In [3]:
from lightweight import lightweight

prompt="""Summarize this text: \n\n\n\n

Lady Gaga (born March 28, 1986) is an American singer, songwriter and actress. Full name Stefani Joanne Angelina Germanotta, Gaga famously dropped out of New York University's Tisch School of the Arts to pursue a career in acting and music as a young adult. She rose to prominence in 2008 with the release of her debut studio album "The Fame," which included the singles "Just Dance" and "Poker Face," both of which hit number one on the Billboard Hot 100 chart. "The Fame" has been certified six-times Platinum and earned an album of the year nomination at the 52nd Annual Grammy Awards. It won best electronic/dance album and best dance recording for its single "Poker Face." The expanded album, "The Fame Monster," released in 2008, cemented Gaga as a pop icon with blockbuster singles "Bad Romance," "Telephone" and "Alejandro." "Bad Romance" alone sold 12 million copies worldwide, becoming one of the best-selling singles of all time.
"""


r=lightweight(prompt)

print(r)



Leading singer Lady Gaga is known for being a versatile artist who combines elements of rock, pop, and R&B into her music. Born on March 31st, 27 years ago, she dropped out from NYU's School of Theater to pursuing a career as an actor before turning towards music full-time. Her breakthrough came in 14th August 2k11 when her first studio album 'The Fame' was released, featuring hits such as 'Just Dance', 'Pokerface', and 'Born This Way'. The album went platinum six times, earning an Album of the Year nomination for Best Electronic/Dance Album at the Gramm
